# 1. Download, Bereinigung und Vorverarbeitung der KfZ-Zulassungsdaten

Einleitung
Der Datensatz KfZ-Zulassung zeigt den jährlichen Bestand an Personenwagen in der Schweiz nach Kanton, Erstzulassungsjahr, Leergewicht, Treibstoff, Hubraum, Antriebs- und Getriebebetrieb, ab 2005.

* [1.1 Erforderliche Python-Libraries](#python_libraries)

* [1.2 KfZ-Daten herunterladen](#data_download)

* [1.3 KfZ-Daten aus Datei laden](#data_load)

* [1.4 Datenbereinigung und Vorverarbeitung](#data_cleansing)
    
* [1.5 Daten in CSV speichern](#data_saving)

#### Datenquelle: https://www.bfs.admin.ch/bfs/de/home/statistiken/kataloge-datenbanken/daten.assetdetail.24105795.html
*Daten heruntergeladen am 01.04.2023*


*Die folgenden Informationen sind der ASTRA-Website entnommen:*

<blockquote>

#### Metainformationen:  
Letzte Änderungen: neue Daten (2022), rektifizierte und revidierte Daten (2021)
Erhebungsdatum: 30. September (ausser 2022: 15. Oktober)
Raumbezug: Kantone am 1.1.2022
Erhebung: Strassenfahrzeugbestand MFZ
Datenquelle: Bundesamt für Strassen (ASTRA) - IVZ-Fahrzeuge, ehemals MOFIS-Datenbank

#### Bemerkungen: 
Die Daten zum Leergewicht und zur Getriebebedienung werden gegenwärtig revidiert. Sie sind daher vorübergehend aus dem Dataset entfernt worden. «Personenwagen» sind leichte Motorwagen zum Personentransport mit höchstens neun Sitzplätzen einschliesslich Fahrer oder Fahrerin. Der Strassenfahrzeugbestand berücksichtigt alle am 30. September in der Schweiz immatrikulierten zivilen Fahrzeuge von Haltern und Halterinnen, die an diesem Stichtag (Wohn-)Sitz in der Schweiz hatten. Aufgrund einer Bereinigung der Grunddatenbank (IVZ-Fahrzeuge) ist die Vergleichbarkeit der Daten ab 2022 mit denen der Vorjahre leicht eingeschränkt: Insgesamt rund 26'000 sich mittlerweile nicht mehr in der Schweiz befindliche Personenwagen mit Export-Kontrollschild wurden 2022 aus der Bestandesstatistik entfernt.

###### Kanton
Der Kanton wird nach der Adresse des Fahrzeughalters oder der -halterin und nicht nach dem Nummernschild bestimmt.
###### Treibstoff
*Benzin*: Bis und mit 2020 inkl. Fahrzeuge ohne Angaben zum Treibstoff (ab 2021 werden diese Fahrzeuge der Kategorie «Anderer» zugewiesen)
\n *Anderer*: Ab 2021 inkl. Fahrzeuge ohne Angaben zum Treibstoff (bis und mit 2020 waren diese Fahrzeuge der Kategorie «Benzin» zugewiesen worden)
</blockquote>

<a id="python_libraries"></a>
## 1.1 Erforderliche Python-Libraries

In [1]:
# Loading necessary libraries in Python
import os
from pathlib import Path
import pandas as pd
import numpy as np

<a id="data_download"></a>
## 1.2 KfZ-Daten herunterladen von bfs.admin.ch 

Die Daten können mit dem unten stehenden Codeblock heruntergeladen werden. Da sich der Link mit der Zeit ändern kann, ist die Ausführung optional, da die Daten bereits in den Ordner data/raw/ heruntergeladen wurden.


In [2]:
! wget https://dam-api.bfs.admin.ch/hub/api/dam/assets/24105795/master

--2023-06-03 18:04:29--  https://dam-api.bfs.admin.ch/hub/api/dam/assets/24105795/master
Resolving dam-api.bfs.admin.ch (dam-api.bfs.admin.ch)... 162.23.128.50
Connecting to dam-api.bfs.admin.ch (dam-api.bfs.admin.ch)|162.23.128.50|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: unspecified [application/octet-stream]
Saving to: ‘master.1’

master.1                [      <=>           ]   4.11M  3.42MB/s    in 1.2s    

2023-06-03 18:04:31 (3.42 MB/s) - ‘master.1’ saved [4306681]



In [3]:
! mv master {raw_data_path}/px-x-1103020100_105.px

mv: cannot move 'master' to '{raw_data_path}/px-x-1103020100_105.px': No such file or directory


<a id="data_load"></a>
## 1.3 Laden von KfZ-Daten aus einer Datei

<div class="alert alert-block alert-info">Die Autodaten sind im .px-Format gespeichert, so dass pyaxis verwendet wird, um die Datei zu lesen und sie als Panda-Datenrahmen zu speichern.</div>

In [4]:
from pyaxis import pyaxis

raw_data_path = Path("./data/raw")
processed_data_path = Path("./data/processed")

kfz_file = "px-x-1103020100_105.px"

# Join the path and file name using the Path object
fp = str(raw_data_path / kfz_file)

#parse contents of *.px file
px = pyaxis.parse(uri = fp , encoding = 'ISO-8859-2')

#store data as pandas dataframe
df_kfz = px['DATA']

# In case the data is to be stored as a dictionary (not necessary in this case, but might be helpful) 
# the following line should be uncommented.  
# meta_dict = px['METADATA']

In [5]:
df_kfz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1935360 entries, 0 to 1935359
Data columns (total 7 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Kanton                        object
 1   Jahr der 1. Inverkehrsetzung  object
 2   Treibstoff                    object
 3   Hubraum                       object
 4   Antrieb                       object
 5   Jahr                          object
 6   DATA                          object
dtypes: object(7)
memory usage: 103.4+ MB


In [6]:
# print the header of the dataframe
df_kfz.head()

,Kanton,Jahr der 1. Inverkehrsetzung,Treibstoff,Hubraum,Antrieb,Jahr,DATA
0,Schweiz,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2005,205
1,Schweiz,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2006,289
2,Schweiz,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2007,279
3,Schweiz,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2008,259
4,Schweiz,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2009,246


<div class="alert alert-block alert-warning">Es sieht so aus, als ob die Spalte *Kanton* andere Werte als die 26 Kantone der Schweiz enthält.</div>

In [7]:
# Getting unique values in column 'Kanton'
df_kfz['Kanton'].unique()

array(['Schweiz', 'Zürich', 'Bern / Berne', 'Luzern', 'Uri', 'Schwyz',
       'Obwalden', 'Nidwalden', 'Glarus', 'Zug', 'Fribourg / Freiburg',
       'Solothurn', 'Basel-Stadt', 'Basel-Landschaft', 'Schaffhausen',
       'Appenzell Ausserrhoden', 'Appenzell Innerrhoden', 'St. Gallen',
       'Graubünden / Grigioni / Grischun', 'Aargau', 'Thurgau', 'Ticino',
       'Vaud', 'Valais / Wallis', 'Neuchâtel', 'Genčve', 'Jura', 'Bund'],
      dtype=object)

<a id="data_cleansing"></a>
## 1.4 Datenbereinigung und Vorverarbeitung

<div class="alert alert-block alert-info">Aus diesem Grund werden alle Zeilen für "Schweiz" und "Bund" aus dem Datenrahmen gelöscht.</div> 

In [8]:
# Dropping all rows with string values of Schweiz and Bund in the Kanton column 
df_kfz = df_kfz[df_kfz['Kanton'].str.contains("Schweiz|Bund") == False]

In [9]:
df_kfz['Kanton'].unique()

array(['Zürich', 'Bern / Berne', 'Luzern', 'Uri', 'Schwyz', 'Obwalden',
       'Nidwalden', 'Glarus', 'Zug', 'Fribourg / Freiburg', 'Solothurn',
       'Basel-Stadt', 'Basel-Landschaft', 'Schaffhausen',
       'Appenzell Ausserrhoden', 'Appenzell Innerrhoden', 'St. Gallen',
       'Graubünden / Grigioni / Grischun', 'Aargau', 'Thurgau', 'Ticino',
       'Vaud', 'Valais / Wallis', 'Neuchâtel', 'Genčve', 'Jura'],
      dtype=object)

<div class="alert alert-block alert-info">
Es gibt doppelte Namen für bestimmte Kantone, in Französisch und Deutsch. Für die Einheitlichkeit der Ergebnisse wird die abgekürzte Version verwendet.</div> 

In [10]:
# Define a dictionary with keys of German/French name and values of abbreviations. The strings in column Kanton 
# is then replaced using this dictionary. 

cantons = {
    'Zürich': 'ZH',
    'Bern / Berne': 'BE',
    'Luzern': 'LU',
    'Uri': 'UR',
    'Schwyz': 'SZ',
    'Obwalden': 'OW',
    'Nidwalden': 'NW',
    'Glarus': 'GL',
    'Zug': 'ZG',
    'Fribourg / Freiburg': 'FR',
    'Solothurn': 'SO',
    'Basel-Stadt': 'BS',
    'Basel-Landschaft': 'BL',
    'Schaffhausen': 'SH',
    'Appenzell Ausserrhoden': 'AR',
    'Appenzell Innerrhoden': 'AI',
    'St. Gallen': 'SG',
    'Graubünden / Grigioni / Grischun': 'GR',
    'Aargau': 'AG',
    'Thurgau': 'TG',
    'Ticino': 'TI',
    'Vaud': 'VD',
    'Valais / Wallis': 'VS',
    'Neuchâtel': 'NE',
    'Genčve': 'GE',
    'Jura': 'JU'
}

df_kfz = df_kfz.replace({'Kanton':cantons})

In [11]:
# Make a check to see if all names have been changed.
df_kfz['Kanton'].unique()

array(['ZH', 'BE', 'LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR', 'SO',
       'BS', 'BL', 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG', 'TI', 'VD',
       'VS', 'NE', 'GE', 'JU'], dtype=object)

In [12]:
# Considering that the Jahr and DATA columns are treated as objects, following transformation are done
df_kfz.Jahr= pd.to_datetime(df_kfz.Jahr, format='%Y').dt.year
df_kfz.DATA = pd.to_numeric(df_kfz.DATA)

In [13]:
# Check to see if the data type has changed
df_kfz.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1797120 entries, 69120 to 1866239
Data columns (total 7 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Kanton                        object
 1   Jahr der 1. Inverkehrsetzung  object
 2   Treibstoff                    object
 3   Hubraum                       object
 4   Antrieb                       object
 5   Jahr                          int64 
 6   DATA                          int64 
dtypes: int64(2), object(5)
memory usage: 109.7+ MB


In [14]:
df_kfz.head()

,Kanton,Jahr der 1. Inverkehrsetzung,Treibstoff,Hubraum,Antrieb,Jahr,DATA
69120,ZH,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2005,37
69121,ZH,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2006,48
69122,ZH,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2007,46
69123,ZH,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2008,40
69124,ZH,Vor 1960,Benzin,Bis 999 cm3,Vorderrad,2009,28


<div class="alert alert-block alert-info">
Für die Analysen in diesem Projekt sind nur die folgenden Spalten von Bedeutung: <b>Kanton</b>, <b>Jahr</b> und <b>Daten</b>, die die Anzahl der Fahrzeuge pro Typ angibt.</div>

In [15]:
# Create a new dataframe with only the above columns. The data is grouped based on the "Kanton" and "Jahr" columns
# and sum of DATA which is number of car per type, is calculated.

df_kfz_grouped = df_kfz.loc[:,['Kanton', 'Jahr', 'DATA']].groupby(['Kanton','Jahr'], as_index=False)['DATA'].sum().rename(columns={'DATA':'Total_KFZ'})
#df_kfz_grouped = df_kfz.groupby(['Kanton','Jahr'])['DATA'].sum()



In [16]:
df_kfz_grouped.head(10)

,Kanton,Jahr,Total_KFZ
0,AG,2005,311370
1,AG,2006,316298
2,AG,2007,321731
3,AG,2008,327074
4,AG,2009,329151
5,AG,2010,339424
6,AG,2011,348117
7,AG,2012,357099
8,AG,2013,364771
9,AG,2014,371223


In [17]:
df_kfz_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Kanton     468 non-null    object
 1   Jahr       468 non-null    int64 
 2   Total_KFZ  468 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 11.1+ KB


<a id="data_saving"></a>
## 1.5 Daten in CSV speichern

<div class="alert alert-block alert-info">
Um sicherzustellen, dass die bisherige Datenverarbeitung funktioniert hat, wird das Ergebnis in einer .csv-Datei gespeichert.</div>

In [18]:
df_kfz_grouped.to_csv(processed_data_path / "interim_kfz_data.csv", sep=";", index=False)